# Overview

This Notebook demonstrates a minimal example of querying the cockroach database using the engine, session, and models defined in the backend code.  The backend code is being read-only volume mounted to `/home/jovyan/backend`, and that directory has been added to the `PYTHONPATH` environment variable.  Imports should "just work".

In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import logging

logging.basicConfig()
logger = logging.getLogger("sqlalchemy.engine")
logger.propagate = True
logger.setLevel(logging.INFO)

<IPython.core.display.Javascript object>

# Show backend engine

In [3]:
import sqlalchemy as sa
import rich

from app.db import engine, db_session
from app.models import UserDAO, TodoDAO

engine

Engine(cockroachdb://root@cockroach:26257/defaultdb?sslmode=disable)

<IPython.core.display.Javascript object>

# Query all Users

In [4]:
with db_session() as session:
    statement = sa.select(UserDAO)
    results = session.execute(statement)
    users = results.scalars().all()

users

INFO:sqlalchemy.engine.Engine:select current_schema()
INFO:sqlalchemy.engine.Engine:[raw sql] {}
INFO:sqlalchemy.engine.Engine:select version()
INFO:sqlalchemy.engine.Engine:[generated in 0.00120s] {}
INFO:sqlalchemy.engine.Engine:SELECT crdb_internal.increment_feature_counter(%(val)s)
INFO:sqlalchemy.engine.Engine:[generated in 0.00072s] {'val': 'sqlalchemy-cockroachdb 1.4.2'}
INFO:sqlalchemy.engine.Engine:SELECT crdb_internal.increment_feature_counter(%(val)s)
INFO:sqlalchemy.engine.Engine:[cached since 0.002703s ago] {'val': 'sqlalchemy 1.4'}
INFO:sqlalchemy.engine.Engine:BEGIN (implicit)
INFO:sqlalchemy.engine.Engine:SELECT users.id, users.created_at, users.name, users.password 
FROM users
INFO:sqlalchemy.engine.Engine:[generated in 0.00209s] {}
INFO:sqlalchemy.engine.Engine:COMMIT


<IPython.core.display.Javascript object>

In [5]:
rich.print(users[0].__dict__)

{
    '_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7f88064918e0>,
    'created_at': datetime.datetime(2021, 11, 6, 20, 23, 36, 942734, 
tzinfo=datetime.timezone.utc),
    'password': 'pass',
    'id': UUID('3f58c723-69c9-4c4d-8dd8-4988d708f31b'),
    'name': 'user3'
}

<IPython.core.display.Javascript object>

In [6]:
for u in users:
    print(u.name)

user3
user2
user1


<IPython.core.display.Javascript object>

# Query all Todos

In [7]:
with db_session() as session:
    statement = sa.select(TodoDAO)
    results = session.execute(statement)
    todos = results.scalars().all()
todos

INFO:sqlalchemy.engine.Engine:BEGIN (implicit)
INFO:sqlalchemy.engine.Engine:SELECT todo.id, todo.created_at, todo.title, todo.content, todo.user_id, users_1.id AS id_1, users_1.created_at AS created_at_1, users_1.name, users_1.password 
FROM todo LEFT OUTER JOIN users AS users_1 ON users_1.id = todo.user_id
INFO:sqlalchemy.engine.Engine:[generated in 0.00093s] {}
INFO:sqlalchemy.engine.Engine:COMMIT


<IPython.core.display.Javascript object>

In [8]:
rich.print(todos[0].__dict__)

{
    '_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7f8806288b50>,
    'content': 'Edit this Note',
    'id': UUID('6eea7c90-706e-445e-83e3-899a0d8a69c1'),
    'created_at': datetime.datetime(2021, 11, 6, 20, 23, 37, 861819, 
tzinfo=datetime.timezone.utc),
    'title': 'Note 2',
    'user_id': UUID('4fba08af-c292-499e-8278-32d46379d9f2'),
    'user': <app.models.UserDAO object at 0x7f8806288b80>
}

<IPython.core.display.Javascript object>